In [32]:
import pandas as pd
import numpy as np

df_raw = pd.read_csv('../RecommenderSystems/data/export_with_basket_2017.csv')

df_raw.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID,SCM_N_ORDER_NR_1
0,341999621,80391417,0,06-JAN-17,1311010,62776372,12378259,1.00,61312,19997,...,139428.16 Royal Canin British Shorthair Adult ...,139428.16 Royal Canin British Shorthair Adult ...,Katzenfutter trocken,Royal Canin Rasse (Breed),British Shorthair,British Shorthair,Royal Canin,Royal Canin,3,80391417
1,348334117,81849313,0,02-FEB-17,2031175,111951408,2913095,1.00,70828,48693,...,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis","523432.4 Bozita Original - 12 kg + 1,8 kg gratis",Hundefutter trocken,Bozita,Bozita,Bozita,Doggy-Bozita,Dry Food Cat / Dog,2,81849313
2,348334114,81849313,0,02-FEB-17,1568093,53626549,2913095,1.00,56851,43068,...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,Hundesnacks,★ Lukullus,Lukullus feine Kauknochen,Lukullus feine Kauknochen,MATINA,snacks,2,81849313
3,354903151,83341895,0,02-MAR-17,1910589,38614848,7097883,0.25,51010,39043,...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3,83341895
4,354903151,83341895,0,02-MAR-17,1910589,38614828,7097883,0.25,51174,39043,...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3,83341895


In [33]:
import pymc3 as pm
import matplotlib.pyplot as plt
%matplotlib inline

## Prepare our dataset to represent a simple transactional one
at the same time let us cast daate to datetime type

In [34]:
transactions = df_raw[['TDT_T_KEY_ORDER_DATE', 
                       'SCM_N_ORDER_NR', 
                       'CUS_N_KEY_CUSTOMER', 
                       'SCM_D_QUANTITY', 
                       'ART_C_PROD_NUMBER']]
transactions['TDT_T_KEY_ORDER_DATE'] = pd.to_datetime(transactions['TDT_T_KEY_ORDER_DATE'])
transactions.head()

,TDT_T_KEY_ORDER_DATE,SCM_N_ORDER_NR,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,ART_C_PROD_NUMBER
0,2017-01-06,80391417,12378259,1.00,139428
1,2017-02-02,81849313,2913095,1.00,523432
2,2017-02-02,81849313,2913095,1.00,468126
3,2017-03-02,83341895,7097883,0.25,445245
4,2017-03-02,83341895,7097883,0.25,445245


In [35]:
## for the sake of readability let us rename the columns

In [38]:
import lifetimes
from lifetimes.datasets import load_dataset
# lifetimes provides a transaction log -> rfm util function# lifetim 
rfm = lifetimes.utils.summary_data_from_transaction_data(
    transactions,
    'CUS_N_KEY_CUSTOMER',
    'TDT_T_KEY_ORDER_DATE',
    observation_period_end=pd.to_datetime('2017-08-30'),
    freq='W'
)

rfm.head()

,frequency,recency,T
CUS_N_KEY_CUSTOMER,,,
22,3.0,31.0,43.0
44,1.0,7.0,51.0
54,0.0,0.0,14.0
64,5.0,35.0,39.0
84,12.0,47.0,50.0


In [43]:
#import HDFStore
#store = HDFStore('store.h5')

#store['df'] = rfm  # save it
#store['df']  # load it

rfm.to_csv('../RecommenderSystems/data/rfm.csv')

In [40]:
len(rfm)

1172020

In [44]:
# import theano 
from pymc3.math import exp, log

class ParetoNBD(pm.Continuous):
    """
    Custom distribution class for Pareto/NBD likelihood.
    """
    
    def __init__(self, lambda_, mu, *args, **kwargs):
        super(ParetoNBD, self).__init__(*args, **kwargs)
        self.lambda_ = lambda_
        self.mu = mu
        
    def logp(self, x, t_x, T):
        """
        Loglikelihood function for and indvidual customer's purchasing rate \lambda
        and lifetime \mu given their frequency, recency and time since first purchase.
        """
        
        log_lambda = log(self.lambda_)
        log_mu = log(self.mu)
        mu_plus_lambda = self.lambda_ + self.mu
        log_mu_plus_lambda = log(mu_plus_lambda)
        
        p_1 = x * log_lambda + log_mu - log_mu_plus_lambda - t_x * mu_plus_lambda
        p_2 = (x + 1) * log_lambda - log_mu_plus_lambda - T * mu_plus_lambda
        
        return log(exp(p_1) + exp(p_2))

In [ ]:
# Extract data for model following notation from Fader/Hardie
N = rfm.shape[0] # number of customers
x = rfm['frequency'].values
t_x = rfm['recency'].values
T = rfm['T'].values # length of training period

n_draws = 2000

pnbd_model = pm.Model()

with pnbd_model:
    
    # Uninformative priors on model hyperparameters see Polson and Scott 
    # https://projecteuclid.org/download/pdfview_1/euclid.ba/1354024466
    r = pm.HalfCauchy('r', beta=2)
    alpha = pm.HalfCauchy('alpha', beta=2)
    s = pm.HalfCauchy('s', beta=2)
    beta = pm.HalfCauchy('beta', beta=2)
    
    # Gamma prior on purchasing rate parameter lambda
    lambda_ = pm.Gamma('lambda', alpha=r, beta=alpha, shape=N, testval=np.random.rand(N))
    # Gamma prior on lifetime parameter mu
    mu = pm.Gamma('mu', alpha=s, beta=beta, shape=N, testval=np.random.rand(N))

    # Custom distribution for Pareto-NBD likelihood function
    loglikelihood = ParetoNBD("loglikelihood", mu=mu, lambda_=lambda_, observed={'x': x, 't_x': t_x, 'T': T})
    
    # Sample the model
    trace = pm.sample(n_draws, init=None)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_log__, lambda_log__, beta_log__, s_log__, alpha_log__, r_log__]
  0%|          | 4/2500 [00:05<59:49,  1.44s/it]  

In [ ]:
# Traceplots to check for convergence
_ = pm.traceplot(trace, varnames=['r','alpha','s','beta'])